In [ ]:
#Libraries required for data importing and data manupulation
import numpy as np
import pandas as pd
import re
#Libraries for preprocessing text data
import nltk
from nltk.corpus import stopwords
from nltk.probability import FreqDist
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

#Libraries for visualisation
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.pyplot as plt


In [ ]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words("english"))
# stop_words.remove('not')
stop_words.update(["ba","flight","British","BA","british","airway", "airline","plane", "told","also","passenger" \
                 "london", "heathrow", "aircraft", "could","even", "would","london","b","u"])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
df_reviews = pd.read_csv('/content/BA_reviews_uncleaned.csv')

df_reviews.head()

,Review,Overall_rating,Type Of Traveller,Seat Type,Route,Seat Comfort,Cabin Staff Service,Food & Beverages,Ground Service,Wifi & Connectivity,Value For Money,Recommended,Date,verified
0,✅ Trip Verified | First time using BA busines...,9,Couple Leisure,Business Class,London to New York,5.0,5.0,5.0,5.0,NaN,4,yes,2024-01-07,True
1,Not Verified | Extremely rude ground service....,6,Family Leisure,Economy Class,Rome to London,4.0,5.0,4.0,1.0,NaN,2,no,2024-01-03,False
2,✅ Trip Verified | My son and I flew to Geneva...,1,Family Leisure,Business Class,Gatwick to Geneva,2.0,1.0,1.0,1.0,1.0,1,no,2024-01-02,True
3,✅ Trip Verified | For the price paid (bought ...,8,Solo Leisure,Business Class,Istanbul to London Heathrow,1.0,4.0,5.0,2.0,NaN,4,yes,2023-12-29,True
4,✅ Trip Verified | Flight left on time and arr...,6,Solo Leisure,Economy Class,London Heathrow to Istanbul,3.0,2.0,1.0,5.0,NaN,2,no,2023-12-29,True


In [ ]:
## To maintain genuineness , we will consider only verified reviews

df_verified_reviews = df_reviews.loc[df_reviews['verified']== True]

df_verified_reviews.head(3)

,Review,Overall_rating,Type Of Traveller,Seat Type,Route,Seat Comfort,Cabin Staff Service,Food & Beverages,Ground Service,Wifi & Connectivity,Value For Money,Recommended,Date,verified
0,✅ Trip Verified | First time using BA busines...,9,Couple Leisure,Business Class,London to New York,5.0,5.0,5.0,5.0,NaN,4,yes,2024-01-07,True
2,✅ Trip Verified | My son and I flew to Geneva...,1,Family Leisure,Business Class,Gatwick to Geneva,2.0,1.0,1.0,1.0,1.0,1,no,2024-01-02,True
3,✅ Trip Verified | For the price paid (bought ...,8,Solo Leisure,Business Class,Istanbul to London Heathrow,1.0,4.0,5.0,2.0,NaN,4,yes,2023-12-29,True


* __We don't need columns like Traveller type,Date,Route and all other ratings__

In [ ]:
df_verified_reviews = df_verified_reviews.drop(['Type Of Traveller','Route','Seat Comfort', 'Cabin Staff Service', 'Food & Beverages',
       'Ground Service', 'Wifi & Connectivity', 'Value For Money','Date'],axis=1)

In [ ]:

df_verified_reviews['Review'] = df_verified_reviews['Review'].str.split('|',expand=True)[1]

df_verified_reviews.head()


,Review,Overall_rating,Seat Type,Recommended,verified
0,First time using BA business class but we we...,9,Business Class,yes,True
2,My son and I flew to Geneva last Sunday for ...,1,Business Class,no,True
3,For the price paid (bought during a sale) it...,8,Business Class,yes,True
4,Flight left on time and arrived over half an...,6,Economy Class,no,True
5,"Very Poor Business class product, BA is not ...",2,Business Class,no,True


In [ ]:
df_verified_reviews.isnull().sum()

Review            0
Overall_rating    0
Seat Type         0
Recommended       0
verified          0
dtype: int64

# Finetuning HuggingFace

In [ ]:
import torch
torch.cuda.is_available()

True

In [ ]:
!pip install datasets transformers huggingface_hub


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.6 MB/s eta 0:00:00


In [ ]:
!apt-get install git-lfs


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.


## Training DistilBERT on IMDB dataset

In [ ]:
from datasets import load_dataset
imdb = load_dataset("imdb")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


* __IMDB is a huge dataset, so let's create smaller datasets to enable faster training and testing:__

In [ ]:
small_train_dataset = imdb["train"].shuffle(seed=42).select([i for i in list(range(5000))])
small_test_dataset = imdb["test"].shuffle(seed=42).select([i for i in list(range(500))])


* __To preprocess our data, you will use DistilBERT tokenizer:__

In [ ]:

from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

* __Will prepare the text inputs for the model for both splits of our dataset (training and test) by using the map method:__

In [ ]:
def preprocess_function(examples):
   return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_train = small_train_dataset.map(preprocess_function, batched=True)
tokenized_test = small_test_dataset.map(preprocess_function, batched=True)


Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

* __To speed up training, let's use a data_collator to convert your training samples to PyTorch tensors and concatenate them with the correct amount of padding:__

In [ ]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


* __let's define DistilBERT as your base model:__

In [ ]:

from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'classifier.weight', 'pre_classifier.bias', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


* __let's define the metrics you will be using to evaluate how good is your fine-tuned model (accuracy and f1 score):__

In [ ]:


import numpy as np
from datasets import load_metric

def compute_metrics(eval_pred):
   load_accuracy = load_metric("accuracy")
   load_f1 = load_metric("f1")

   logits, labels = eval_pred
   predictions = np.argmax(logits, axis=-1)
   accuracy = load_accuracy.compute(predictions=predictions, references=labels)["accuracy"]
   f1 = load_f1.compute(predictions=predictions, references=labels)["f1"]
   return {"accuracy": accuracy, "f1": f1}

### Hugging Face API login

* __Define the training arguments and define a Trainer with all the objects you constructed up to this point:__

In [ ]:
from transformers import TrainingArguments, Trainer

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
! pip install -U accelerate
! pip install -U transformers

In [ ]:
training_args = TrainingArguments(
   output_dir='test_trainer',
   learning_rate=2e-5,
   per_device_train_batch_size=16,
   per_device_eval_batch_size=16,
   num_train_epochs=3,
   weight_decay=0.01,
   save_strategy="epoch",
   push_to_hub= True
)

In [ ]:

trainer = Trainer(
   model=model,
   args=training_args,
   train_dataset=tokenized_train,
   eval_dataset=tokenized_test,
   tokenizer=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

Step,Training Loss
500,0.295000


TrainOutput(global_step=939, training_loss=0.2210528665203383, metrics={'train_runtime': 745.956, 'train_samples_per_second': 20.108, 'train_steps_per_second': 1.259, 'total_flos': 1987010979840000.0, 'train_loss': 0.2210528665203383, 'epoch': 3.0})

In [ ]:
trainer.evaluate()


/usr/local/lib/python3.10/dist-packages/datasets/load.py:752: FutureWarning: The repository for accuracy contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.1/metrics/accuracy/accuracy.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:752: FutureWarning: The repository for f1 contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.1/metrics/f1/f1.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next majo

{'eval_loss': 0.30949243903160095,
 'eval_accuracy': 0.9,
 'eval_f1': 0.9007936507936508,
 'eval_runtime': 10.0311,
 'eval_samples_per_second': 49.845,
 'eval_steps_per_second': 3.19,
 'epoch': 3.0}

## Pushing trained model to hugging face hub

In [ ]:
trainer.push_to_hub()

events.out.tfevents.1705494507.9a8df3b8ef37.3459.5:   0%|          | 0.00/457 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Swam80/test_trainer/commit/42faf84cfc8189aed7446e21eb51c830ddbd75ac', commit_message='End of training', commit_description='', oid='42faf84cfc8189aed7446e21eb51c830ddbd75ac', pr_url=None, pr_revision=None, pr_num=None)

# Using pushed model for Sentiment Analysis

In [ ]:
from transformers import pipeline


distil_bert_pipeline = pipeline("text-classification", model="Swam80/test_trainer")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [ ]:
df_bert_senti = df_verified_reviews.copy()

In [ ]:
df_bert_senti.head(2)

,Review,Overall_rating,Seat Type,Recommended,verified,Review_processed
0,First time using BA business class but we we...,9,Business Class,yes,True,first time using business class pleased servic...
2,My son and I flew to Geneva last Sunday for ...,1,Business Class,no,True,son flew geneva last sunday skiing holiday le ...


In [ ]:
def apply_sentiment(review):
    try:
        # Truncate the review to the maximum sequence length
        if len(review.split()) > 512:
            review = ' '.join(review.split()[:512])
        result = distil_bert_pipeline(review)[0]
        # Map the labels
        sentiment = 'Positive' if result['label'] == 'LABEL_1' else 'Negative'
        return pd.Series([sentiment, result['score']])
    except RuntimeError as e:
        print(f"Error with review: {review}")
        print(str(e))

In [ ]:
 df_bert_senti[['Sentiment', 'Score']] = df_bert_senti['Review'].apply(apply_sentiment)

Error with review: Much better experience this time around. Flew CPT to LHR on a new A350. Checkin was efficient. I used the Bidvest lounge at CPT as it is better than BA's lounge. Boarding was speedy. The new business class club suites are very good. Slightly irritating to be directed to the wrong seat (1A instead of 1K) which meant that after settling in the passenger in 1A showed up. We arranged a seat switch. Second time in a row it has happened. Why can't BA staff memorise which side of the plane is which and get it right? Captain's announcement was short and on point. Welcome drink was served at the correct temperature i.e. cold (first time ever for me - previously BA has always served the welcome champagne warm). IFE system was good - everything worked as it should. Felt slightly cheap that business class passengers have to pay for wifi. I did not bother. Cocktail was good. The nuts came on a cheap plastic tray which was naff ... other airlines bring out something classier. A fe

TypeError: object of type 'NoneType' has no len()

# We can see that, few reviews have exceeded the token limit of 512. We will use text division method where a big text will be tokenized and divided into two parts.